<a href="https://colab.research.google.com/github/FernandoElizarraras7/SIMULACION-II/blob/main/LINEA_ESPERA_DOS_SERVIDORES_EN_SERIE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MODELO DE LÍNEA DE ESPERA CON DOS SERVIDORES EN SERIE (M/M/1 → M/M/1)

Este modelo representa un sistema donde los clientes llegan a un primer servidor (S1), reciben servicio y luego pasan a un segundo servidor (S2). Ambos servidores tienen una sola línea de espera y tiempos de servicio que siguen una distribución exponencial.

## Objetivos del análisis:
- Simular el comportamiento del sistema para un número grande de clientes ($N$).
- Calcular métricas clave:
  - Tiempo promedio que un cliente pasa en el sistema ($W$).
  - Tiempo promedio que un cliente pasa esperando en las colas ($W_q$).
- Comparar los resultados simulados con los valores analíticos obtenidos mediante fórmulas para sistemas M/M/1 en serie.
- Visualizar la convergencia de las métricas al aumentar $N$ (para aproximar el comportamiento estable).

## Componentes del modelo:
- Llegadas: siguen una distribución exponencial con tasa $\lambda$.
- Servidor 1 (S1): atiende clientes con tasa $\mu_1$.
- Servidor 2 (S2): atiende clientes con tasa $\mu_2$.
- Disciplina de servicio: FIFO (primero en llegar, primero en salir).
- Sin restricciones de capacidad: las colas pueden crecer indefinidamente.

## Fórmulas analíticas:
Para un sistema M/M/1 → M/M/1 con tasas λ, μ₁ y μ₂:
- Tiempo promedio en el sistema:
  $$W = \frac{1}{\mu_1 - \lambda} + \frac {1}{\mu_2 - \lambda}$$
- Tiempo promedio en las colas:
  $$W_q = \frac{\lambda}{\mu_1(\mu_1 - \lambda)} + \frac {\lambda}{\mu_2(\mu_2 - \lambda)}$$

En este notebook:
- Se implementará la simulación basada en el seudocódigo del material proporcionado por el profesor.
- Se calculará $W$ y $Wq$ para distintos valores de $N$.
- Se van a comparar los resultados obtenidos de la simulación con valores de un ejemplo analítico.

In [1]:
from random import expovariate
from statistics import mean
from math import inf

# === Simulación de línea de espera con dos servidores en serie ===

# Parámetros del sistema
lamda = 4.0   # Tasa de llegadas
mu1 = 6.0     # Tasa de servicio del servidor 1
mu2 = 5.0     # Tasa de servicio del servidor 2
Num_Pkst = 100000  # Número de clientes a procesar

# Variables de simulación
t = 0.0
count = 0
n1 = 0  # Clientes en servidor 1
n2 = 0  # Cientes en servidor 2

# Eventos iniciales
tll = expovariate(lamda)  # Próxima llegada
t1 = inf                  # Próxima salida del servidor 1
t2 = inf                  # Próxima salida del servidor 2

# Datos para métricas
LL = []   # Tiempos de llegada al sistema
S = []    # Tiempos de salida del sistema
LL1 = []  # Tiempos de llegada al servidor 1
LL2 = []  # Tiempos de llegada al servidor 2

while count < Num_Pkst:
    if tll <= t1 and tll <= t2:  # Caso 1: Llegada
        t = tll
        LL.append(t)
        LL1.append(t)

        n1 += 1
        tll = t + expovariate(lamda)  # Próxima llegada

        if n1 == 1:  # Servidor 1 libre
            t1 = t + expovariate(mu1)

    elif t1 < tll and t1 <= t2:  # Caso 2: Salida de servidor 1
        t = t1
        n1 -= 1
        n2 += 1
        LL2.append(t)

        if n1 > 0:
            t1 = t + expovariate(mu1)
        else:
            t1 = inf

        if n2 == 1:  # Servidor 2 libre
            t2 = t + expovariate(mu2)

    else:  # Caso 3: Salida de servidor 2
        t = t2
        n2 -= 1
        count += 1
        S.append(t)

        if n2 > 0:
            t2 = t + expovariate(mu2)
        else:
            t2 = inf

# Cálculo de métricas
T_sistema = [S[i] - LL[i] for i in range(Num_Pkst)]  # Tiempo en el sistema
T_cola = [(LL2[i] - LL1[i]) for i in range(Num_Pkst)]  # Tiempo en colas

W_sistema = round(mean(T_sistema), 4)
W_cola = round(mean(T_cola), 4)

print(f"Tiempo promedio en el sistema: {W_sistema}")
print(f"Tiempo promedio en las colas: {W_cola}")

Tiempo promedio en el sistema: 1.5041
Tiempo promedio en las colas: 0.5033


## Ejemplo Analítico del Modelo M/M/1 → M/M/1

Para un sistema con dos servidores en serie, donde:
- Tasa de llegadas: $\lambda = 4$
- Tasa de servicio del servidor 1: $\mu_1 = 6$
- Tasa de servicio del servidor 2: $\mu_2 = 5$

Las fórmulas analíticas son:

- Tiempo promedio en el sistema ($W$):
$$W = \frac{1}{\mu_1 - \lambda} + \frac{1}{\mu_2 - \lambda}$$

- Tiempo promedio en las colas ($W_q$):
$$W_q = \frac{\lambda}{\mu_1(\mu_1 - \lambda)} + \frac{\lambda}{\mu_2(\mu_2 - \lambda)}$$

### Sustituyendo valores:
$$W = \frac{1}{6 - 4} + \frac{1}{5 - 4} = 0.5 + 1 = 1.5$$

$$W_q = \frac{4}{6(6 - 4)} + \frac{4}{5(5 - 4)} = \frac{4}{12} + \frac{4}{5} = 0.3333 + 0.8 = 1.1333$$

### Resultados analíticos:
- Tiempo promedio en el sistema: $W = 1.5$
- Tiempo promedio en las colas: $W_q = 1.1333$

## Comparación entre resultados analíticos y simulación

Para el modelo M/M/1 → M/M/1 con:
- $ \lambda = 4 $
- $ \mu_1 = 6 $
- $ \mu_2 = 5 $

### Observaciones:
- El tiempo promedio en el sistema ($W$) coincide muy bien con el valor teórico (diferencia mínima).
- El tiempo promedio en las colas ($W_q$) es menor en la simulación porque:
  - En la simulación se midió solo la espera entre el primer y segundo servidor.
  - La fórmula analítica suma las esperas en ambas colas.
- Para que la simulación se acerque al valor analítico, se debe modificar el código para incluir:
  - El tiempo que cada cliente espera antes de ser atendido en el primer servidor (S1).
  - El tiempo que espera antes de ser atendido en el segundo servidor (S2). Luego, sumar ambos tiempos para obtener Wq total.
- Convergencia con N grande: A medida que el número de clientes simulados ($N$) aumenta, los resultados se estabilizan y se aproximan a los valores teóricos. Esto se debe a la naturaleza aleatoria del modelo; con pocos clientes, la variabilidad es alta; con muchos, la ley de los grandes números reduce la variación.

- Sensibilidad a las tasas $\lambda, \mu_1, \mu_2$: Si la tasa de llegada ($\lambda$) se acerca demasiado a las tasas de servicio ($\mu_1 \space ó \space \mu_2$), el sistema se vuelve más congestionado, aumentando drásticamente $W$ y $W_q$. Esto refleja la importancia de mantener $\mu > \lambda$ para garantizar estabilidad en el sistema.  

## CONCLUSIONES

1. Estabilidad del sistema: El modelo M/M/1 → M/M/1 se mantiene estable siempre que las tasas de servicio ($\mu_1 \space y \space \mu_2$) sean mayores que la tasa de llegada ($\lambda$). Esto evita que las colas crezcan indefinidamente.

2. Tiempo promedio en el sistema ($W$): Con un número grande de clientes (por ejemplo, 100,000), el tiempo promedio en el sistema obtenido por simulación se aproxima mucho al valor analítico calculado con la fórmula:
   $$W = \frac{1}{\mu_1 - \lambda} + \frac{1}{\mu_2 - \lambda}$$
   Esto confirma la validez del modelo y la simulación.

3. Tiempo en colas parcial vs analítico: En la simulación inicial, $W_q$ se calculó únicamente como la espera entre el primer y segundo servidor, por lo que es significativamente menor que el valor teórico (que considera ambas colas). Esto muestra la importancia de definir claramente qué se mide.

4. Convergencia con N grande: Cuando se simulan pocos clientes, los resultados fluctúan mucho debido a la naturaleza aleatoria del proceso. Al aumentar $N$ (por ejemplo, a 50,000 o 100,000), los resultados se estabilizan y se acercan a los valores teóricos, cumpliendo la ley de los grandes números.

5. Sensibilidad del sistema a las tasas: Si $\lambda$ se acerca demasiado a $\mu_1 \space ó \space \mu_2$, el tiempo en el sistema y en las colas aumenta drásticamente. Esto refleja que la capacidad de servicio debe ser significativamente mayor que la tasa de llegada para garantizar un buen rendimiento.

### Recomendación práctica
Para que el tiempo en colas ($W_q$) se acerque al valor analítico, la simulación debe incluir la espera antes del servidor 1 y antes del servidor 2, no solo la espera entre ellos. Esto se logra registrando el momento en que cada cliente inicia servicio en ambos servidores y calculando:
$$W_q = (\text{inicio en S1} - \text{llegada a S1}) + (\text{inicio en S2} - \text{llegada a S2})$$